# Vérification de signature d'emails avec Python
Dans cette Tech Review, nous présentons les différentes solutions étudiées pour vérifier des
emails signés avec Python.  

## Import des librairies


In [46]:
import email
import subprocess
from pprint import pprint

from asn1crypto import cms
from cryptography.hazmat.bindings._rust import test_support
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.ec import ECDSA, EllipticCurvePublicKey
from cryptography.hazmat.primitives.serialization import (
    Encoding,
    load_pem_private_key,
    pkcs12,
    PrivateFormat,
    NoEncryption,
)
from cryptography.hazmat.primitives.serialization.pkcs7 import (
    PKCS7Options,
    PKCS7SignatureBuilder,
)
from cryptography.x509 import load_pem_x509_certificate
from endesive import email as endesive_email

## Création d'un nouveau certificat

In [47]:
# Lecture d'un nouveau certificat sous format pkcs12
with open("vectors/verify/verify.p12", "rb") as f:
    private_key, certificate, _ = pkcs12.load_key_and_certificates(f.read(), b"1234")

# Écriture du certificat en PEM
with open("vectors/verify/verify.pem", "wb") as f:
    f.write(certificate.public_bytes(Encoding.PEM))

# Ecriture de la clé privée en PEM
with open("vectors/verify/verify_key.pem", "wb") as f:
    f.write(
        private_key.private_bytes(
            Encoding.PEM, PrivateFormat.TraditionalOpenSSL, NoEncryption()
        )
    )

## Lecture du certificat X.509

In [3]:
# Clé publique : certificat RSA
with open("vectors/verify/ca.pem", "rb") as file:
    certificate = load_pem_x509_certificate(file.read())
type(certificate)

cryptography.hazmat.bindings._rust.x509.Certificate

In [4]:
# Clé publique du certificat
public_key = certificate.public_key()
print("Type:", type(public_key))
isinstance(public_key, EllipticCurvePublicKey)

Type: <class 'cryptography.hazmat.bindings._rust.openssl.ec.ECPublicKey'>


True

In [5]:
# Clé privée : RSA
with open("vectors/verify/ca_key.pem", "rb") as file:
    private_key = load_pem_private_key(file.read(), password=None)

## Signature d'un email

In [6]:
# Écriture d'un message non-chiffré
# message = b"Hello world!\n"
message = b"Hello world!"
canonical_message = b"Content-Type: text/plain\r\n\r\n" + message
with open("vectors/message.txt", "wb") as file:
    file.write(message)
with open("vectors/canonical_message.txt", "wb") as file:
    file.write(canonical_message)

### Avec `openssl`

In [48]:
# SMIME : ancienne version
instructions = [
    "openssl",
    "smime",  # on peut utiliser smime ou cms
    "-sign",
    "-in",
    "vectors/message.txt",
    "-out",
    "vectors/verify/signed.txt",
    "-signer",
    "vectors/verify/verify.pem",
    "-inkey",
    "vectors/verify/verify_key.pem",
    "-noattr",  # optionnel
    "-text",  # optionnel
]
smime_encryption_result = subprocess.run(instructions, check=True, capture_output=True)
smime_encryption_result.stderr

b''

In [49]:
# Signature
instructions = [
    "openssl",
    "smime",  # on peut utiliser smime ou cms
    "-sign",
    "-in",
    "vectors/message.txt",
    "-out",
    "vectors/verify/signature.der",
    "-outform",
    "DER",
    "-signer",
    "vectors/verify/verify.pem",
    "-inkey",
    "vectors/verify/verify_key.pem",
    "-noattr",  # optionnel
    "-text",  # optionnel
]
smime_encryption_result = subprocess.run(instructions, check=True, capture_output=True)
smime_encryption_result.stderr

b''

### Avec `cryptography`
Les fonctionnalités de chiffrage d'email sont disponibles dans la librairie `cryptography` depuis
début 2024. 

In [27]:
# Prepare the envelope builder
builder = PKCS7SignatureBuilder()
builder = builder.add_signer(certificate, private_key, hashes.SHA256())
builder = builder.set_data(message)

# Encrypt the data
options = [
    # PKCS7Options.DetachedSignature,
    # PKCS7Options.NoAttributes,
    # PKCS7Options.Text,
]
signed = builder.sign(Encoding.SMIME, options)
signature = builder.sign(Encoding.DER, options)

In [28]:
# Store the data in vectors
with open("vectors/verify/signed.txt", "wb") as file:
    file.write(signed)
with open("vectors/verify/signature.der", "wb") as file:
    file.write(signature)

## Solutions abordées

### Solution 1: avec `openssl`

#### Structure ASN.1

In [8]:
instructions = [
    "openssl",
    "asn1parse",
    "-in",
    "vectors/verify/signature.der",
    "-inform",
    "der",
]
output = subprocess.run(instructions, check=True, capture_output=True)
print(output.stdout.decode())

    0:d=0  hl=4 l= 738 cons: SEQUENCE          
    4:d=1  hl=2 l=   9 prim: OBJECT            :pkcs7-signedData
   15:d=1  hl=4 l= 723 cons: cont [ 0 ]        
   19:d=2  hl=4 l= 719 cons: SEQUENCE          
   23:d=3  hl=2 l=   1 prim: INTEGER           :01
   26:d=3  hl=2 l=  15 cons: SET               
   28:d=4  hl=2 l=  13 cons: SEQUENCE          
   30:d=5  hl=2 l=   9 prim: OBJECT            :sha256
   41:d=5  hl=2 l=   0 prim: NULL              
   43:d=3  hl=2 l=  27 cons: SEQUENCE          
   45:d=4  hl=2 l=   9 prim: OBJECT            :pkcs7-data
   56:d=4  hl=2 l=  14 cons: cont [ 0 ]        
   58:d=5  hl=2 l=  12 prim: OCTET STRING      :Hello world!
   72:d=3  hl=4 l= 343 cons: cont [ 0 ]        
   76:d=4  hl=4 l= 339 cons: SEQUENCE          
   80:d=5  hl=3 l= 249 cons: SEQUENCE          
   83:d=6  hl=2 l=   3 cons: cont [ 0 ]        
   85:d=7  hl=2 l=   1 prim: INTEGER           :02
   88:d=6  hl=2 l=   2 prim: INTEGER           :0309
   92:d=6  hl=2 l=  10 cons: 

#### Vérification
Ici, on doit rajouter "-noverify" pour ne pas vérifier la signature du certificat, car sinon on a
l'erreur suivante: "self-signed certificate". Nous n'avons pas cette erreur en utilisant la fonction `test_support.pkcs7_verify` de la librairie `cryptography`.

In [56]:
# Message signé
instructions = [
    "openssl",
    "smime",
    "-verify",
    "-in",
    "vectors/verify/signed.txt",
    # "-certfile",
    # "vectors/verify/verify.pem",
    # "-CAfile",
    # "vectors/verify/verify_ca.pem",
    "-noverify",
]
output = subprocess.run(instructions, capture_output=True)
output.stderr.decode()

'Verification successful\r\n'

In [10]:
# Message signé
instructions = [
    "openssl",
    "smime",
    "-verify",
    "-in",
    "vectors/verify/signature.der",
    "-inform",
    "der",
    "-content",
    "vectors/message.txt",
    "-certfile",
    "vectors/verify/ca.pem",
    "-noverify",
]
output = subprocess.run(instructions, capture_output=True)
output.stderr.decode()

'Verification successful\r\n'

### Solution 2 : avec `asn1crypto`

In [19]:
with open("vectors/verify/signature.der", "rb") as file:
    signature = file.read()

#### Structure ASN.1

In [20]:
# Load the structure
content_info = cms.ContentInfo.load(signature)
content_type: cms.ContentType = content_info["content_type"]
signed_data: cms.SignedData = content_info["content"]

# # About the content type
content_type.native, content_type.dotted

('signed_data', '1.2.840.113549.1.7.2')

In [21]:
def native_to_json(native_content):
    if isinstance(native_content, list):
        return [native_to_json(item) for item in native_content]
    elif isinstance(native_content, dict):
        return {key: native_to_json(value) for key, value in native_content.items()}
    else:
        return native_content

In [22]:
# About the signed data
# -- Version
version: cms.CMSVersion = signed_data["version"]
print("Version:", version.native)

# -- Digest algorithms
digest_algorithms: cms.DigestAlgorithms = signed_data["digest_algorithms"]
for i, digest_algorithm in enumerate(digest_algorithms):
    print(f"\nDigest algorithm {i+1}:", digest_algorithm["algorithm"].native)

# -- Encapsulated content info
encap_content_info: cms.EncapsulatedContentInfo = signed_data["encap_content_info"]
print("Encapsulated content info:", native_to_json(encap_content_info.native))

# -- Certificates
certificates: cms.CertificateSet = signed_data["certificates"]
for i, cert in enumerate(certificates):
    print(f"\nCertificate {i+1}:")
    pprint(native_to_json(cert.native))

# -- Signer infos
signer_infos: cms.SignerInfos = signed_data["signer_infos"]
for i, signer_info in enumerate(signer_infos):
    print(f"\nSigner info {i+1}:")
    pprint(native_to_json(signer_info.native))

Version: v1

Digest algorithm 1: sha256
Encapsulated content info: {'content_type': 'data', 'content': None}

Certificate 1:
{'signature_algorithm': {'algorithm': 'sha256_ecdsa', 'parameters': None},
 'signature_value': b'0F\x02!\x00\xbd\xb6\x19K\xf1G\xcc\xdbK\x18l\xad\x1d\x13U'
                    b'\r\xd70-b\x12X\xe0s\xea\x9fY\xbd.\x82\xe3k\x02!\x00'
                    b'\xd3\x85\x1d\xb6\x0e\xe6\x19=C\x10\xdf\xf0\x04\xca\\X'
                    b'\x9c\xc3~\xc8\x11\xe2QG\xd4\xee\x10\xef\xf8\x82\xafL',
 'tbs_certificate': {'extensions': [{'critical': True,
                                     'extn_id': 'basic_constraints',
                                     'extn_value': {'ca': True,
                                                    'path_len_constraint': None}}],
                     'issuer': {'common_name': 'cryptography CA',
                                'country_name': 'US'},
                     'issuer_unique_id': None,
                     'serial_number': 777,
        

In [23]:
# Zoom sur le seul signer info
signer_info = signed_data["signer_infos"][0]

# Les données à l'intérieur
version: cms.CMSVersion = signer_info["version"]
sid: cms.SignerIdentifier = signer_info["sid"]
digest_algorithm: cms.DigestAlgorithm = signer_info["digest_algorithm"]
signature_algorithm: cms.SignedDigestAlgorithm = signer_info["signature_algorithm"]
signature_data: cms.OctetString = signer_info["signature"]

# Signed attributes
signed_attrs: cms.CMSAttributes = signer_info["signed_attrs"]
for i, attribute in enumerate(signed_attrs):
    print(f"\nSigned attribute {i+1}:")
    print(attribute)
    pprint(native_to_json(attribute.native))

#### Vérification de la signature
Il faut vérifier la signature en utilisant le certificat (et pas la clé privée !)

In [24]:
public_key: EllipticCurvePublicKey = certificate.public_key()
if signed_attrs.native is None: 
    # verify_data = message
    verify_data = b"Content-Type: text/plain\r\n\r\n" + message
else:
    verify_data = signed_attrs.dump()
    verify_data = b"\x31" + verify_data[:1]  # 0x31 is the SEQUENCE tag
public_key.verify(signature_data.native, verify_data, ECDSA(hashes.SHA256())) 

### Solution 3 : avec `cryptography` !
Il est possible de vérifier la signature d'un mail avec une fonction rust qui appelle les fonctions
OpenSSL. Cette fonction est privée et n'est donc pas censée être utilisée par les utilisateurs de la
librairie. D'où l'intérêt de construire une API publique ! 

In [32]:
# Récupération du contenu - message.replace fonctionne aussi
signed_message = email.message_from_bytes(signed)
content = signed_message.get_payload()[0].get_payload(decode=True)

# Vérification du message signé
test_support.pkcs7_verify(Encoding.SMIME, signed, content, [certificate], [])

In [33]:
# Vérification de la signature
test_support.pkcs7_verify(Encoding.DER, signature, canonical_message, [certificate], [])

### Solution 4 : avec `endesive`
La librairie `endesive` permet de vérifier la signature d'un email. Elle est basée sur les
librairies `asn1crypto`, `certvalidator` et `cryptography`.

In [31]:
endesive_email.verify(signed.decode())

********** failed certificate verification: Unable to build a validation path for the certificate "Email Address: demo1@trisoft.com.pl" - no issuer matching "Common Name: CA" was found
cert.issuer: OrderedDict([('common_name', 'CA')])
cert.subject: OrderedDict([('email_address', 'demo1@trisoft.com.pl')])


(True, True, False)